In [1]:
import pandas as pd
import numpy as np 

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from datetime import datetime
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.set_seed = 42


train_feature = np.load('Data/tdcsfog/train_feature.npy')
valid_feature = np.load('Data/tdcsfog/valid_feature.npy')
test_feature = np.load('Data/tdcsfog/test_feature.npy')
train_label = np.load('Data/tdcsfog/train_label.npy')
valid_label = np.load('Data/tdcsfog/valid_label.npy')
test_label = np.load('Data/tdcsfog/test_label.npy')


2023-05-09 09:11:51.019444: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_feature.shape, valid_feature.shape, train_label.shape, valid_label.shape, test_feature.shape, test_label.shape

((12454643, 3),
 (3113661, 3),
 (12454643, 4),
 (3113661, 4),
 (3892076, 3),
 (3892076, 4))

In [3]:
train_feature, valid_feature, test_feature

(array([[-0.44120461, -0.082227  , -1.21183151],
        [-0.44090003, -1.08226973,  0.45736664],
        [-0.60049354, -0.53963709, -1.42609465],
        ...,
        [ 0.44363822,  1.50481888,  2.06660539],
        [ 0.53577687, -0.8985712 ,  1.23031254],
        [ 0.5300372 , -1.09046895,  0.04516791]]),
 array([[ 1.21631488,  0.8841906 ,  0.54111506],
        [ 0.02318879, -0.05761225,  1.06627182],
        [-0.34960191,  0.3380232 ,  0.65026524],
        ...,
        [ 0.0700669 ,  0.3553626 , -0.5310615 ],
        [-0.0840419 ,  0.89741266, -2.15641887],
        [ 1.02971993, -0.21155021,  1.35195849]]),
 array([[-0.1616217 ,  0.58656796, -0.09556338],
        [-1.1087879 , -0.57619191, -0.81476793],
        [ 0.56795388, -0.3863001 ,  0.74809813],
        ...,
        [-0.16639802,  0.27430714, -0.51293622],
        [-1.09601198, -2.18623926,  1.12397463],
        [-0.43500182,  0.17089489, -0.41409062]]))

In [4]:
# Building the model and pipeline for Tensorflow 
# Preprocess the data
# Turn our data into TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_feature, train_label))
train_dataset =  train_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_feature,valid_label))
valid_dataset = valid_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_feature,test_label))
test_dataset = test_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)

2023-05-09 09:11:52.871375: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 09:11:52.890538: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 09:11:52.890739: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
# path = 'Data/defog/'
# train_dataset.save(path+'train_dataset')
# valid_dataset.save(path+'valid_dataset')
# test_dataset.save(path+'test_dataset')

In [6]:
# path = 'Data/defog/'
# train_dataset = tf.data.Dataset.load(path+'train_dataset')
# valid_dataset = tf.data.Dataset.load(path+'valid_dataset')
# test_dataset =  tf.data.Dataset.load(path+'test_dataset')
# test_feature = np.load('Data/defog/test_feature.npy')
# test_label = np.load('Data/defog/test_label.npy')

In [7]:
# Check the data shape
# print(
#     f"Train_feature : {train_feature.shape} {train_feature.dtype} \n" 
#     f"Train_labels : {train_label.shape} {train_label.dtype} \n" 
#     f"Valid_feature : {valid_feature.shape} {valid_feature.dtype} \n" 
#     f"Valid_label : {valid_label.shape} {valid_label.dtype} "
#     ) 

print(f"Train : {train_dataset} \n"
      f"Valid : {valid_dataset} \n" 
      f"Test : {test_dataset}")

Train : <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))> 
Valid : <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))> 
Test : <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.int64, name=None))>


In [8]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [9]:
# Build the model 
epoch = 10
input_shape = 3

inputs = layers.Input(shape=input_shape, name="input_layer")
x = layers.Dense(32, activation ='relu')(inputs)
x = layers.Dense(64, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.1)(x)
# x = layers.Dense(128, activation ='relu')(x)
# x = layers.Dense(512, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.25)(x)
# x = layers.Dense(1024, activation ='relu')(x)
# x = layers.Dense(512, activation = 'relu')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation="relu")(x)
# x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax",name="output_layer")(x)      
model = tf.keras.Model(inputs, outputs) 

model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 32)                128       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 output_layer (Dense)        (None, 4)                 132       
                                                                 
Total params: 4,452
Trainable params: 4,452
Non-trainable params: 0
_________________________________________________________________


In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report

In [11]:
import numpy as np
np.random.seed(40)
from mlflow import MlflowClient
from mlflow.entities import ViewType
from mlflow.models.signature import infer_signature

# !mlflow server --backend-store-uri sqlite:///backend.db

In [12]:
def eval_metrics(classifier, test_features, test_labels):
    
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    
    # make prediction
    model_preds_probs   = classifier.predict(test_features)
    model_preds         = tf.argmax(model_preds_probs, axis=1)
    predictions         = tf.argmax(test_labels,axis=1)

    
    # Calculate model accuracy
    # model_accuracy = accuracy_score(test_labels, predictions) * 100
    # Calculate model precision, recall and f1 score using "weighted average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(test_labels, 
                                                                                 predictions, 
                                                                                 average="weighted", 
                                                                                 zero_division= 1)
    
    av_precision    = average_precision_score(test_labels, predictions, zero_division= 1)
    
    model_results = {
        # "accuracy": model_accuracy,
        "precision": model_precision,
        "recall": model_recall,
        "f1": model_f1,
        "av precisoin" : av_precision}
    
    
    
    
    
    # target_names = ['StartHesitation','Turn','Walking', 'All_zero']
    # print("Classification report")
    # print("---------------------","\n")
    # print(classification_report(test_labels, predictions,target_names=target_names),"\n")
    # print("Confusion Matrix")
    # print("---------------------","\n")
    # print(f"{Matrix} \n")

    print("Accuracy Measures")
    print("---------------------","\n")
    # print("Accuracy: ", model_accuracy)
    print("Avarge Precision: ", av_precision)
    print("precision: ", model_precision)
    print("recall : ", model_recall)
    print("f1 : ", model_f1)
    
    return model_results

In [13]:
import mlflow
import mlflow.tensorflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://127.0.0.1:5000'


In [14]:
import tensorflow as tf
# how to check tensorflow version
tf.__version__
len(tf.config.list_physical_devices('GPU:0'))>0

False

In [15]:
from sklearn.model_selection import cross_val_score
counter = 1
Model_Name = 'DNN_model'
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.set_seed = 42
np.random.seed(40)

with mlflow.start_run():
    mlflow.tensorflow.autolog()
    
    # fit the model
    # with tf.device('/GPU:0'):
    start = datetime.now()
    history_model = model.fit(train_dataset,
                            batch_size=2048,
                            steps_per_epoch=len(train_dataset),
                            validation_data=valid_dataset,
                            validation_steps=int(len(valid_dataset)),
                            callbacks=[early_stopping, reduce_lr],
                            epochs=epoch) 
    end = datetime.now()
    print(f"The time taken to train the model is {end - start}")
        
    # Evaluate model
    model.evaluate(test_dataset)
    
    # Calculate the metrics
    # model_results = eval_metrics(model, 
    #                              test_features = test_feature,
    #                              test_labels = test_label)
    
    mlflow.log_param("Model"           , Model_Name)
    mlflow.log_param("Dataset ", "tdcsfog")
    # mlflow.log_params(model_results)
    mlflow.tensorflow.autolog()
    
    
    print("________________________________________")

Epoch 1/10


2023-05-09 09:12:05.437232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [12454643,4]
	 [[{{node Placeholder/_1}}]]
2023-05-09 09:12:07.020635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-09 09:12:07.022617: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x5636a064a980 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-09 09:12:07.022643: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-05-09 09:12:07.027744: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash repr

6081/6082 [============================>.] - ETA: 0s - loss: 1.1565 - accuracy: 0.4689

2023-05-09 09:12:25.426354: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [3113661,4]
	 [[{{node Placeholder/_1}}]]


6082/6082 [==============================] - 24s 4ms/step - loss: 1.1565 - accuracy: 0.4689 - val_loss: 1.1424 - val_accuracy: 0.4778 - lr: 0.0010
Epoch 2/10
6082/6082 [==============================] - 21s 3ms/step - loss: 1.1411 - accuracy: 0.4778 - val_loss: 1.1381 - val_accuracy: 0.4798 - lr: 0.0010
Epoch 3/10
6082/6082 [==============================] - 21s 3ms/step - loss: 1.1383 - accuracy: 0.4795 - val_loss: 1.1366 - val_accuracy: 0.4804 - lr: 0.0010
Epoch 4/10
6082/6082 [==============================] - 21s 3ms/step - loss: 1.1367 - accuracy: 0.4802 - val_loss: 1.1355 - val_accuracy: 0.4808 - lr: 0.0010
Epoch 5/10
6082/6082 [==============================] - 21s 3ms/step - loss: 1.1357 - accuracy: 0.4807 - val_loss: 1.1348 - val_accuracy: 0.4811 - lr: 0.0010
Epoch 6/10
6082/6082 [==============================] - 21s 3ms/step - loss: 1.1349 - accuracy: 0.4811 - val_loss: 1.1342 - val_accuracy: 0.4814 - lr: 0.0010
Epoch 7/10
5844/6082 [===========================>..] - ETA: 0s